<a href="https://colab.research.google.com/github/BunnyNoBugs/coursework-semantic-roles/blob/main/rubert_rlc_error_detection_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install transformers

     |████████████████████████████████| 1.8MB 21.3MB/s 
     |████████████████████████████████| 3.2MB 32.3MB/s 
     |████████████████████████████████| 890kB 57.0MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=a2ea282fc4a38abd5bc1dd163f9e37673fdc42bc96828a7225e30f6e020fe27b
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
import torch
from transformers import AutoTokenizer, AutoModel, BertForSequenceClassification, Trainer, TrainingArguments

In [ ]:
device = 'cuda:0'

In [ ]:
rlc_dev = pd.read_csv('https://raw.githubusercontent.com/BunnyNoBugs/coursework-semantic-roles/main/rlc_dev.csv', index_col=[0])
rlc_test = pd.read_csv('https://raw.githubusercontent.com/BunnyNoBugs/coursework-semantic-roles/main/rlc_test.csv', index_col=[0])

In [ ]:
rlc_dev['label'][rlc_dev['label'] == 'correct'] = 0
rlc_dev['label'][rlc_dev['label'] == 'incorrect'] = 1
rlc_train, rlc_val = train_test_split(rlc_dev, random_state=42)

In [ ]:
rlc_train

,text,label
31232,"Согласно Конституции России, республики входящ...",0
44633,Для меня русский язык означает гораздо больше ...,1
53111,"И здесь не только может пит кофе, но тоже слуш...",1
33693,Мы все летом бываем у них на даче и она сама н...,1
45300,Я это говорю всерьёз: она придерживается очень...,0
...,...,...
38829,"В программе подробно описаны основные пункты, ...",0
52002,"Я думаю, что если говоришь о людях, да, очень,...",0
50047,в начале (?) моих планов я хочу серьёзно работ...,0
11846,"будет выглядеть лучше, так хорошая страна!",0


In [ ]:
tokenizer = AutoTokenizer.from_pretrained("DeepPavlov/rubert-base-cased")

In [ ]:
train_encodings = tokenizer(list(rlc_train['text']), truncation=True, padding=True)
val_encodings = tokenizer(list(rlc_val['text']), truncation=True, padding=True)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [ ]:
class RLCDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels
    
    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['label'] = torch.tensor(self.labels[idx])

        return item

    def __len__(self):
        return len(self.labels)

train_dataset = RLCDataset(train_encodings, list(rlc_train['label']))
val_dataset = RLCDataset(val_encodings, list(rlc_val['label']))

In [ ]:
model = BertForSequenceClassification.from_pretrained("DeepPavlov/rubert-base-cased", num_labels=2)
model.to(device)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at DeepPavlov/rubert-base-cased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elemen

In [ ]:
training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=3,              # total number of training epochs
    per_device_train_batch_size=16,  # batch size per device during training
    per_device_eval_batch_size=64,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=10,
)

trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=val_dataset             # evaluation dataset
)

trainer.train()

Step,Training Loss
10,0.703000
20,0.704000
30,0.703800
40,0.684800
50,0.669500
60,0.679300
70,0.665400
80,0.646700
90,0.638400
100,0.669800


Step,Training Loss
10,0.703000
20,0.704000
30,0.703800
40,0.684800
50,0.669500
60,0.679300
70,0.665400
80,0.646700
90,0.638400
100,0.669800


TrainOutput(global_step=3000, training_loss=0.4462520456314087, metrics={'train_runtime': 3063.0577, 'train_samples_per_second': 0.979, 'total_flos': 17248946325209424, 'epoch': 3.0})

In [ ]:
model.save_pretrained('/content/drive/MyDrive/Colab Notebooks/Курсовая по семантическим ролям/rubert_rlc_error_detection')

In [ ]:
a = tokenizer('Этим летом я роеду один месяц в португалие', return_tensors='pt')
a.to(device)

{'input_ids': tensor([[  101, 10742, 15105,   877, 41560, 63101,  5259, 16293,   845, 19243,
          2343,   842,   102]], device='cuda:0'), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], device='cuda:0')}

In [ ]:
torch.softmax(model(**a).logits, dim=1)

tensor([[0.9964, 0.0036]], device='cuda:0', grad_fn=<SoftmaxBackward>)

In [ ]:
*tokenizer('asdf')

SyntaxError: ignored